# How to Migrate minto.v0 Experiment to minto.v1

This notebook is a guide to help you migrate your minto.v0 experiment to minto.v1.

In [1]:
import minto
import numpy as np
import jijmodeling as jm

In [2]:
tsp = minto.problems.tsp.QuadTSP()
tsp_problem = tsp.problem()
tsp_data = tsp.data(n=8)

## minto.v0 experiment

You can use the `v0.x.x` version of the minto in `v1.x.x` with `minto.v1`.
In the following, we create a simple experiment using `minto.v0` and then migrate it to `minto.v1`.

In [50]:
import jijmodeling_transpiler as jmt

compiled_instance = jmt.core.compile_model(tsp_problem, tsp_data)
qubo_builder = jmt.core.pubo.transpile_to_pubo(compiled_instance)
qubo, _ = qubo_builder.get_qubo_dict()

In [51]:
import minto.v0
import openjij as oj

sampler = oj.SASampler()
sweeps_list = np.arange(100, 1000, 100)

exp_v0 = minto.v0.Experiment("v0_exp")
for sweeps in sweeps_list:
    with exp_v0.run():
        response = sampler.sample_qubo(qubo, num_reads=10, num_sweeps=sweeps)
        sampleset = jmt.core.pubo.openjij_decode.decode_from_openjij(
            response,
            qubo_builder,
            compiled_instance
        )
        sampleset = jm.experimental.from_old_sampleset(sampleset)
        exp_v0.log_result("result", sampleset)
        exp_v0.log_parameter("sweeps", sweeps)


In [52]:
exp_v0.table(enable_sampleset_expansion=False)

,experiment_name,run_id,sweeps,result
0,v0_exp,0,100,"(Sample(run_id=""4b9b8389-a331-4ddd-a8d7-a9afbd..."
1,v0_exp,1,200,"(Sample(run_id=""380588c0-e95c-474a-911a-5dd2ed..."
2,v0_exp,2,300,"(Sample(run_id=""2e84fa06-08b5-4818-9b39-1f7e06..."
3,v0_exp,3,400,"(Sample(run_id=""6cf101e2-0673-488a-a21f-2f7dd4..."
4,v0_exp,4,500,"(Sample(run_id=""c02d1bc0-8535-4ecd-beb1-770e04..."
5,v0_exp,5,600,"(Sample(run_id=""13ee025e-99dc-4c2e-aefa-aed347..."
6,v0_exp,6,700,"(Sample(run_id=""2ca17c58-8b0e-4604-ada9-3110ec..."
7,v0_exp,7,800,"(Sample(run_id=""a55c41cb-1ecf-4df2-8d0c-3af435..."
8,v0_exp,8,900,"(Sample(run_id=""11207a98-f05f-4376-ad49-9a84c6..."


## Migrate from minto.v0 to minto.v1

To migrate from `minto.v0` to `minto.v1`, you need to use `migrate_to_v1_from_v0` function.

In [53]:
from minto.migrator.v0tov1 import migrate_to_v1_from_v0

exp_v1 = migrate_to_v1_from_v0(exp_v0)

[2025-08-01 22:35:29] 🚀 Starting experiment 'v0_exp'
[2025-08-01 22:35:29]   ├─ 📊 Environment: OS: Linux 6.6.93+, CPU: Intel(R) Xeon(R) CPU @ 2.80GHz (4 cores), Memory: 15.6 GB, Python: 3.11.10
[2025-08-01 22:35:29]   ├─ 📊 Environment Information
[2025-08-01 22:35:29]       ├─ OS: Linux 6.6.93+
[2025-08-01 22:35:29]       ├─ Platform: Linux-6.6.93+-x86_64-with-glibc2.35
[2025-08-01 22:35:29]       ├─ CPU: Intel(R) Xeon(R) CPU @ 2.80GHz (4 cores)
[2025-08-01 22:35:29]       ├─ Memory: 15.6 GB
[2025-08-01 22:35:29]       ├─ Architecture: x86_64
[2025-08-01 22:35:29]       ├─ Python: 3.11.10
[2025-08-01 22:35:29]       ├─ Key Package Versions:
[2025-08-01 22:35:29]   ├─ 🏃 Created run #0
[2025-08-01 22:35:29]       ├─ 📝 Parameter: sweeps = 100
[2025-08-01 22:35:29]       ├─ 📊 SampleSet 'result': 0 samples
[2025-08-01 22:35:29]   ├─ ✅ Run #0 completed (0.1s)
[2025-08-01 22:35:29]   ├─ 🏃 Created run #1
[2025-08-01 22:35:29]       ├─ 📝 Parameter: sweeps = 200
[2025-08-01 22:35:29]       ├─ 📊 

In [54]:
exp_v1.get_run_table()

sampleset_result                                                   \
            num_samples  obj_mean   obj_std   obj_min   obj_max feasible   
run_id                                                                     
0                    10  2.703292  0.271523  2.362134  3.326194       10   
1                    10  2.851875  0.347497  2.466858  3.553749       10   
2                    10  2.638969  0.639893  1.452414  3.518039        4   
3                    10  2.609291  0.195442  2.341296  3.037372       10   
4                    10  2.307913  0.680801  0.975667  3.268000        1   
5                    10  2.618542  0.413288  2.049099  3.277035        4   
6                    10  2.781062  0.317559  2.281351  3.230759       10   
7                    10  2.405925  0.673386  1.297302  3.662775        2   
8                    10  2.710594  0.301820  2.281351  3.202480       10   

               parameter metadata               
          name    sweeps   run_id elapsed_time  
run_id                                          
0       result       100        0     0.055799  
1       result       200        1     0.004895  
2       result       300        2     0.004403  
3       result       400        3     0.004015  
4       result       500        4     0.003958  
5       result       600        5     0.004461  
6       result       700        6     0.004591  
7       result       800        7     0.004686  
8       result       900        8     0.004513

Then you can save the migrated experiment using `.save()` or `.save_as_ommx_archive` function.

In [55]:
# exp_v1.save()  # save the experiment at disk
# exp_v1.save_as_ommx_archive("v1_exp.ommx")  # save the experiment as an ommx archive